# sklearnの`Pipeline`と`make_pipe_line`の紹介
## Agenda

- `sklearn.pipeline.Pipeline`のsyntaxと使用方法の紹介

## 参考

- [`sklearn.pipeline.Pipeline`のcache機能の紹介](https://blog.amedama.jp/entry/scikit-learn-pipeline-cache)

## `sklearn.pipeline.Pipeline`
### 機能
- preprocessingの一連の処理(feature selection, normalization)と最終的に用いるestimatorをつなげるパイプライン（一連の処理を実行してくれるオブジェクト, class）を作ってくれる
- Observed data, X, しかtransformできない
- nameをautomaticallyにlowercaseで記述してくれる簡易コンストラクターが`sklearn.pipeline.make_pipeline`

### When to Use

1. 前処理とestimatorの一連の流れを一つのオブジェクトにまとめるため
2. grid search
3. leakageの予防のため

### Params
- steps: list

List of tuples. 実行したいtransformや`fit()`の対象となるestimatorをプロセスの名前と共に格納する。名前を`key`, stepを`value` としたとき、tupleの記述は(`key`, `value`)となる。
```
[('scaler', StandardScaler()), ('svc', SVC())]
```

- memory: str or object with the joblib.Memory interface, default=None

Pipeline の最終段に位置するモデル (Estimator) に渡される直前のデータをキャッシュしてくれる

- verbose: bool, default=False

<img src = 'https://github.com/RyoNakagami/omorikaizuka/blob/master/algorithm/sklearn_pipeline_method.jpg?raw=true'>

### Example

In [1]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
X, y = make_classification(random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    random_state=0)
pipe = Pipeline([('scaler', StandardScaler()), ('svc', SVC())])
# The pipeline can be used as any other estimator
# and avoids leaking the test set into the train set
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

0.88

## Pipeline 機能のキャッシュの使い方

- `Pipeline` クラスのコンストラクタで `memory` オプションに`joblib.memory.Memory` オブジェクトを渡さなくてはならない

### Example

- Digits データセットを主成分分析して、それをランダムフォレストで識別する

In [2]:
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.decomposition import KernelPCA
from sklearn.model_selection import train_test_split
from joblib import memory

In [3]:
dataset = datasets.load_digits()
X, y = dataset.data, dataset.target
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    shuffle=True, 
                                                    test_size=0.33,
                                                    random_state=42)

Pipeline constructor

In [4]:
steps = [
        # 主成分分析
        ('pca', KernelPCA()),
        # ランダムフォレスト
        ('rf', RandomForestClassifier(n_estimators=100))
    ]
cache = memory.Memory(location='./cachedir', verbose=3)
pipeline = Pipeline(steps=steps, memory=cache)

In [5]:
pipeline.fit(X_train, y_train)

[Memory]0.0s, 0.0min    : Loading _fit_transform_one...


Pipeline(memory=Memory(location=./cachedir/joblib),
         steps=[('pca', KernelPCA()), ('rf', RandomForestClassifier())])

In [6]:
y_pred = pipeline.predict(X_test)
acc = accuracy_score(y_pred, y_test)
print(f'accuracy: {acc}')

accuracy: 0.9629629629629629


## Usage: `sklearn.pipeline.Pipeline`

### PipelineのConstruction

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.decomposition import PCA
estimators = [('reduce_dim', PCA()), ('clf', SVC())]
pipe = Pipeline(estimators)
pipe

Pipeline(steps=[('reduce_dim', PCA()), ('clf', SVC())])

### Accessing steps

- `idx`または`name`でpipelineのstepにアクセスすることができる

In [8]:
pipe[0]

PCA()

In [9]:
pipe['reduce_dim']

PCA()

### Grid search

`<estimator>__<parameter>`でpipelineのstepのパラメータ設定することができる

In [10]:
from sklearn.model_selection import GridSearchCV
param_grid = dict(reduce_dim__n_components=[2, 5, 10],
                  clf__C=[0.1, 10, 100])
grid_search = GridSearchCV(pipe, param_grid=param_grid, cv = 5)
grid_search.fit(X, y)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('reduce_dim', PCA()), ('clf', SVC())]),
             param_grid={'clf__C': [0.1, 10, 100],
                         'reduce_dim__n_components': [2, 5, 10]})

最もよかった際の学習のパラメータの表示

In [11]:
grid_search.best_estimator_

Pipeline(steps=[('reduce_dim', PCA(n_components=10)), ('clf', SVC(C=10))])